In [1]:
import experiment_framework as ef
import fast_model_improved as fmi
from pyexpat import model
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import copy

In [2]:
G = nx.erdos_renyi_graph(3000, 0.005)
model = ef.run_influence_cascade(G, num_influencers=50, steps=3, selection_method='random')

# Evaluate the influence and deinfluence
num_influenced = model.evaluate_influence()
num_deinfluenced = model.evaluate_deinfluence()
num_susceptible = model.evaluate_susceptible()

# Create a deep copy of the model
model_fixed_influencers = copy.deepcopy(model)

# Evaluate the influence and deinfluence
num_influenced = model_fixed_influencers.evaluate_influence()
num_deinfluenced = model_fixed_influencers.evaluate_deinfluence()
num_susceptible = model_fixed_influencers.evaluate_susceptible()

print(f"Number of influenced nodes: {num_influenced}")
print(f"Number of deinfluenced nodes: {num_deinfluenced}")
print(f"Number of susceptible nodes: {num_susceptible}")
print('influencer', model.selected_influencers)

Number of influenced nodes: 2983
Number of deinfluenced nodes: 0
Number of susceptible nodes: 17
influencer {1272, 897, 641, 1157, 1030, 1927, 1545, 1033, 1165, 1933, 1677, 145, 1657, 1046, 2457, 2843, 285, 545, 546, 2468, 1661, 1192, 1962, 2358, 1593, 697, 2877, 2368, 960, 1600, 1732, 1348, 72, 2894, 2002, 2643, 1235, 2262, 1632, 2528, 1122, 2403, 2667, 1647, 1520, 755, 376, 2553, 2300, 893}


In [3]:
# Run simulation for different values of k
k_deinfluencers_ls = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]  # Different values of k to test
deinfluencers_list = ef.select_deinfluencers(k_deinfluencers_ls, model_fixed_influencers)
results = ef.average_results(deinfluencers_list, model_fixed_influencers, num_runs=3, steps=3)

ValueError: Sample larger than population or is negative

In [ ]:
# Define different marker styles for each method
marker_styles = {
    'Random': 'o',
    'RanExIniInf': 's',
    'RanExAllInf': 'D',
    'Degree': 'v',
    'Closeness': '^',
    'Betweenness': '<',
    'Eigenvector': '>',
    'PageRank': 'P',
    'RIniInf': '*',
    'RInfl': 'h',
    'RRankedIniInf': 'X',
    'RRankedInf': 'd'
}

# Plotting results
fig, axs = plt.subplots(3, figsize=(20, 20))
axs[0].set_title('Measured Deinfluence against Deinfluencer selection methods')
axs[1].set_title('Measured Influence against Deinfluencer selection methods')
axs[2].set_title('Measured Susceptible against Deinfluencer selection methods')

# Create line plots
methods = results[next(iter(results))].keys()  # Get all methods from the first key
k_values = sorted(results.keys())  # Sort k values for plotting

total_nodes = len(G.nodes)

for method in methods:
    deinfluenced_nodes = [results[k][method][0] for k in k_values]
    influenced_nodes = [results[k][method][1] for k in k_values]
    susceptible_nodes = [total_nodes - (influenced + deinfluenced) for influenced, deinfluenced in zip(influenced_nodes, deinfluenced_nodes)]

    marker = marker_styles.get(method, 'o')  # Default to 'o' if method is not in marker_styles

    axs[0].plot(k_values, deinfluenced_nodes, label=method, marker=marker)
    axs[1].plot(k_values, influenced_nodes, label=method, marker=marker)
    axs[2].plot(k_values, susceptible_nodes, label=method, marker=marker)

axs[0].legend()
axs[0].set_xlabel('Number of Deinfluencers')
axs[0].set_ylabel('Average Number of Deinfluenced Nodes')

axs[1].legend()
axs[1].set_xlabel('Number of Deinfluencers')
axs[1].set_ylabel('Average Number of Influenced Nodes')

axs[2].legend()
axs[2].set_xlabel('Number of Deinfluencers')
axs[2].set_ylabel('Average Number of Susceptible Nodes')

plt.tight_layout()
plt.show()
